In [1]:
#imports
import nltk


# custom functions
from helper import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\apoor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
filename = "../books/alice_adventure_new_lines_reduced.txt"
data = getSents(filename)

In [4]:
data

['\ufeffALICE’S ADVENTURES IN WONDERLAND  Lewis Carroll  THE MILLENNIUM FULCRUM EDITION 3.0     CHAPTER I.',
 'Down the Rabbit-Hole  Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, ‘and what is the use of a book,’ thought Alice ‘without pictures or conversations?’  So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.',
 'There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, ‘Oh dear!',
 'Oh dear!',
 'I shall be late!’ (when she thought it over afterwards, it occurred to her that she ought to have wondered at t

In [3]:
data

['\ufeffALICE’S ADVENTURES IN WONDERLAND$1Lewis Carroll$1THE MILLENNIUM FULCRUM EDITION 3.0$1CHAPTER I.',
 'Down the Rabbit-Hole$1Alice was beginning to get very tired of sitting by her sister on the$1bank, and of having nothing to do: once or twice she had peeped into the$1book her sister was reading, but it had no pictures or conversations in$1it, ‘and what is the use of a book,’ thought Alice ‘without pictures or$1conversations?’$1So she was considering in her own mind (as well as she could, for the$1hot day made her feel very sleepy and stupid), whether the pleasure$1of making a daisy-chain would be worth the trouble of getting up and$1picking the daisies, when suddenly a White Rabbit with pink eyes ran$1close by her.$1There was nothing so VERY remarkable in that; nor did Alice think it so$1VERY much out of the way to hear the Rabbit say to itself, ‘Oh dear!$1Oh dear!',
 'I shall be late!’ (when she thought it over afterwards, it$1occurred to her that she ought to have wondered at 